# ExoNAMD API v1.0

## Summary

The observer wants to compute the relative and/or absolute NAMD of:
- a given multiplanetary system;
- a subset of multiplanetary systems;
- or all the known ones.

This tool handles all of the above.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from datetime import timedelta

pd.options.display.max_columns = 20
pd.options.display.max_rows = 30
pd.options.mode.copy_on_write = True
import warnings

warnings.filterwarnings("ignore")

from exonamd.utils import ROOT
from exonamd.utils import fetch_aliases
from exonamd.utils import update_host
from exonamd.utils import update_planet

### Task 1: getting the data

This task retrieves the parameters of confirmed systems from the NASA Exoplanet Archive database, and stores them in a local database.

In [3]:
# Define the URL for the API
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"

# Define the date you want to filter by
from_scratch = False  # change to False in production
if from_scratch:
    latest = datetime.strptime('1950-01-01', "%Y-%m-%d")  # Example date
else:
    df_old = pd.read_csv(os.path.join(ROOT, "data", "exo.csv"))
    latest = df_old['rowupdate'].max()
    latest = datetime.strptime(latest, "%Y-%m-%d")  # Example date
    latest = latest - timedelta(days=1)

# Convert the date to a string in the format 'YYYY-MM-DD'
latest = latest.strftime('%Y-%m-%d')

# Define the multiplicity you want to filter by
multiplicity = 1

# Define the SQL query to retrieve the required data
query = f"""
SELECT 
    hostname, 
    pl_name, 
    default_flag,
    rowupdate,
    sy_pnum, 
    pl_orbsmax, 
    pl_orbsmaxerr1, 
    pl_orbsmaxerr2, 
    pl_rade,
    pl_radeerr1,
    pl_radeerr2,
    pl_bmasse, 
    pl_bmasseerr1, 
    pl_bmasseerr2, 
    pl_orbeccen, 
    pl_orbeccenerr1, 
    pl_orbeccenerr2, 
    pl_orbincl, 
    pl_orbinclerr1, 
    pl_orbinclerr2,
    pl_trueobliq,
    pl_trueobliqerr1,
    pl_trueobliqerr2
FROM ps
WHERE
    sy_pnum > '{multiplicity}'
    AND rowupdate > '{latest}'
"""

# Define the parameters for the request
params = {
    "query": query,
    "format": "json",
}

# Make the request to the API
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

### Task 2: dealing with the aliases

Fetch aliases

In [4]:
aliases = fetch_aliases(df['hostname'].unique())

Fetched 966/966 entries on try 1


Curate aliases

In [6]:
# # For testing
# df2 = df.copy()
# df2.loc[df2['hostname'] == 'Kepler-20', 'hostname'] = 'KOI-70'
# df2.loc[df2['hostname'] == 'Kepler-80', 'hostname'] = 'KIC 4852528'
# df2['hostname'] = df2.apply(update_host, args=(aliases, True), axis=1)
df['hostname'] = df.apply(update_host, args=(aliases, True), axis=1)

# # For testing
# df2 = df.copy()
# df2.loc[df2['pl_name'] == 'Kepler-20 c', 'pl_name'] = 'KOI-70.01'
# df2.loc[df2['pl_name'] == 'Kepler-11 b', 'pl_name'] = 'KOI-157 b'
# df2['pl_name'] = df2.apply(update_planet, args=(aliases, True), axis=1)
df['pl_name'] = df.apply(update_planet, args=(aliases, True), axis=1)

Found TOI-1338 b in aliases, updating to TOI-1338 A b
Found TOI-1338 b in aliases, updating to TOI-1338 A b
Found TOI-1338 b in aliases, updating to TOI-1338 A b
Found TOI-1338 c in aliases, updating to TOI-1338 A c
Found TOI-1338 b in aliases, updating to TOI-1338 A b
Found TOI-1338 c in aliases, updating to TOI-1338 A c
Found HIP 29442 c in aliases, updating to HD 42813 c
Found pi Men c in aliases, updating to HD 39091 c
Found pi Men d in aliases, updating to HD 39091 d
Found pi Men c in aliases, updating to HD 39091 c
Found pi Men c in aliases, updating to HD 39091 c
Found HD 11343 c in aliases, updating to HIP 8541 c
Found pi Men c in aliases, updating to HD 39091 c
Found Kepler-448 c in aliases, updating to KOI-12 c
Found pi Men c in aliases, updating to HD 39091 c
Found pi Men c in aliases, updating to HD 39091 c
Found TOI-411 b in aliases, updating to HD 22946 b
Found TOI-411 c in aliases, updating to HD 22946 c
Found TOI-411 b in aliases, updating to HD 22946 b
Found TOI-411 c 

In [7]:
for hostname in df["hostname"].unique():
    df_host = df[df["hostname"] == hostname]
    # we check the names of the planets in the system
    # if they use a different naming convention, we discard the system
    names = df_host["pl_name"]
    # check if the first three characters are the same
    if len(set([name[:3] for name in names])) > 1:
        # discard the system
        print(f"Discarding {hostname}")

### Task 3: computing missing values (if any) from simple equations

### Task 4: storing the curated database

In [8]:
if not from_scratch:
    df_new = df.copy()
    df = pd.concat([df_new, df_old], ignore_index=True)
    df = df.drop_duplicates(keep='last')

df.to_csv(os.path.join(ROOT, "data", "exo.csv"), index=False)

In [9]:
df[df["pl_trueobliq"].notnull()][["pl_name", "pl_trueobliq", "pl_trueobliqerr1", "pl_trueobliqerr2"]]

,pl_name,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2
482,K2-290 c,124.00,6.00,-6.00
728,HAT-P-11 b,106.00,15.00,-11.00
939,XO-2 N b,27.00,12.00,-27.00
1191,WASP-107 b,109.81,28.17,-13.64
1202,V1298 Tau b,8.00,4.00,-7.00
1205,Kepler-9 b,28.10,13.00,-13.60
1404,WASP-84 b,17.30,7.70,-7.70
3327,Kepler-25 c,26.90,7.00,-9.20
4473,TOI-942 b,2.00,27.00,-23.00
5105,Kepler-462 b,72.00,3.00,-3.00


### Task 5: input missing values (if any) by interpolation

In [10]:
# code here

### Task 6: storing the curated+interpolated database

In [11]:
# code here